In [17]:
import pandas as pd
import numpy as np
from datetime  import datetime  
from datetime import timedelta  
import feather
import lightgbm as lgb
from sklearn import preprocessing, metrics


In [18]:
X_train = pd.read_pickle('../01_preprocessed_data/X_train.pkl')
y_train = pd.read_pickle('../01_preprocessed_data/y_train.pkl')
X_val = pd.read_pickle('../01_preprocessed_data/X_val.pkl')
y_val = pd.read_pickle('../01_preprocessed_data/y_val.pkl')
X_test = pd.read_pickle('../01_preprocessed_data/X_test.pkl')
submission = pd.read_csv('../00_data/sample_submission.csv')

In [19]:
#X_train = feather.read_dataframe('../01_preprocessed_data/X_train.feather')
#y_train = feather.read_dataframe('../01_preprocessed_data/y_train.feather')
#X_val = feather.read_dataframe('../01_preprocessed_data/X_val.feather')
#y_val = feather.read_dataframe('../01_preprocessed_data/y_val.feather')
#X_test = feather.read_dataframe('../01_preprocessed_data/X_test.feather')
#submission = pd.read_csv('../00_data/sample_submission.csv')

In [20]:
#X_train = X_train.drop(columns = ['date', 'start_date'])
#X_val = X_val.drop(columns = ['date', 'start_date'])
#X_test = X_test.drop(columns = ['date', 'start_date'])

In [21]:
#cat = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
#for feature in cat:
#    encoder = preprocessing.LabelEncoder()
#    X_train[feature] = encoder.fit_transform(X_train[feature])
#    X_val[feature] = encoder.fit_transform(X_val[feature])
#    X_test[feature] = encoder.fit_transform(X_test[feature])

In [22]:
#X_train = X_train.drop(columns = ['avg_month'])
#X_val = X_val.drop(columns = ['avg_month'])
#X_test= X_test.drop(columns = ['avg_month'])

In [48]:
X_test.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
       'items_sold', 'date', 'weekday', 'month', 'event_name_1',
       'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX',
       'snap_WI', 'start_date', 'days_from_start', 'start_date_from_start',
       'sell_price', 'avg_weekday', 'avg_month', 'avg_price'],
      dtype='object')

In [49]:
params = {
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'objective': 'regression',
        'n_jobs': -1,
        'seed': 0,
        'learning_rate': 0.1,
        'bagging_fraction': 0.75,
        'bagging_freq': 10, 
        'colsample_bytree': 0.75}
features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday', 'month', 
            'event_name_1', 'event_type_1','event_name_2', 'event_type_2',
            'snap_CA', 'snap_TX', 'snap_WI',
            'days_from_start', 'start_date_from_start', 'sell_price',
           'avg_weekday', # 'avg_month', 
            'avg_price']
train_set = lgb.Dataset(X_train[features], y_train)
val_set = lgb.Dataset(X_val[features], y_val)

In [50]:
X_train[features].columns

Index(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday',
       'month', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'days_from_start',
       'start_date_from_start', 'sell_price', 'avg_weekday', 'avg_price'],
      dtype='object')

In [ ]:
#del x_train, y_train

model = lgb.train(params, train_set, num_boost_round = 2500, early_stopping_rounds = 50, 
                  valid_sets = [train_set, val_set], verbose_eval = 100)

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 2.75018	valid_1's rmse: 2.38765
[200]	training's rmse: 2.67912	valid_1's rmse: 2.32461
[300]	training's rmse: 2.63594	valid_1's rmse: 2.30761
[400]	training's rmse: 2.60412	valid_1's rmse: 2.29149
[500]	training's rmse: 2.57988	valid_1's rmse: 2.27865
[600]	training's rmse: 2.55811	valid_1's rmse: 2.27061


In [ ]:
val_pred = model.predict(X_val[features])
val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
print(f'Our val rmse score is {val_score}')
y_test = model.predict(X_test[features])
X_test['items_sold'] = y_test


In [ ]:
X_test['items_sold'] = y_test
X_test

In [ ]:
#submission = pd.read_csv('../00_data/sample_submission.csv')

In [ ]:
X_test

In [ ]:
predictions = X_test[['id', 'date', 'items_sold']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'items_sold').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]

validation = submission[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])
final

In [ ]:
final.to_csv('../04_submissions/lightGBM_no_fe.csv', index = False)

In [ ]:
features.append(str(val_score))

In [ ]:
with open("../04_submissions/lgb_features_score.txt", "a") as outfile:
    outfile.write("\n".join(features))
